<a href="https://colab.research.google.com/github/subraja1810/BERT/blob/Master/CS18B1059!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install bert-for-tf2


In [13]:
import tensorflow as tf
from tensorflow import keras

In [14]:
!pip install transformers
!pip install praw

In [33]:
from transformers import TFAutoModelForTokenClassification, AutoTokenizer 
from matplotlib import pyplot as plt 
import numpy as np
import praw 
#import pandas as pd 

In [34]:
model = TFAutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english") 
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

All model checkpoint layers were used when initializing TFBertForTokenClassification.

All the layers of TFBertForTokenClassification were initialized from the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


In [35]:
label_list = [
    "O",       # Outside of a named entity
    "B-MISC",  # Beginning of a miscellaneous entity immediately after another miscellaneous entity
    "I-MISC",  # Miscellaneous entity
    "B-PER",   # Beginning of a person's name immediately after another person's name
    "I-PER",   # Person's name
    "B-ORG",   # Beginning of an organisation immediately after another organisation
    "I-ORG",   # Organisation
    "B-LOC",   # Beginning of a location immediately after another location
    "I-LOC"    # Location
]

In [36]:
reddit = praw.Reddit(client_id='3PQwAJT-jv_t2g', 
                     client_secret='Lj2yqyiO-r5RO-K-7NAM7ABx9mL4qg',
                     user_agent='RedditBoo')

In [37]:
def replies_of(top_comments, comment_list):
    if len(top_comments.replies) == 0:
        return
    else:
        for num, comment in enumerate(top_comments.replies):
            try:
                comments_list.append(str(comment.body))
            except:
                continue
            replies_of(comment, comments_list)

In [ ]:
count = 0
final_dict = {'I-LOC': [], 'I-ORG': [], 'I-PER': [], 'B-LOC': [], 'B-ORG': [], 'B-PER': [], 'B-MISC': [], 'I-MISC': []} 
word_temp = ''
current_tag = ''
old_tag = ''
print_dict = {}
print(final_dict)

In [39]:
list_of_subreddit = ['marvelstudios']

In [ ]:
for j in list_of_subreddit:
    top_posts = reddit.subreddit(j).top('month', limit=1)
    comments_list = [] #Data fetched.
    # save subreddit comments in dataframe
    for submission in top_posts:
        submission_comm = reddit.submission(id=submission.id)
        comments_list.append(str(submission.title))

        for count, top_comments in enumerate(submission_comm.comments):
            try:
                replies_of(top_comments, comments_list)
            except:
                continue

print(comments_list)                

In [41]:
for sequence in comments_list:
    if len(sequence) > 512: #BERT token limitaton.
        continue
    # Standard Tokenization
    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
    inputs = tokenizer.encode(sequence, return_tensors="tf")
    outputs = model(inputs)[0]
    predictions = tf.argmax(outputs, axis=2)
    #print(predictions)
    # Tokenized input
    list_bert = [(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].numpy())]
    for i in list_bert:
        if i[1] in ['O']:
            if len(current_tag) > 0:
                without_space_word = word_temp.strip()
                if len(without_space_word) > 1:
                    final_dict[current_tag].append(without_space_word)
            count = 0
            word_temp = ''
            current_tag = ''
            continue
        else:
            current_tag = i[1]

            if old_tag != current_tag and len(old_tag) > 0:
                without_space_word = word_temp.strip()
                if len(without_space_word) > 1:
                    final_dict[old_tag].append(without_space_word)
                count = 0
                word_temp = ''
                current_tag = ''

            if i[0].startswith('##'): 
                word_temp += i[0][2:].upper()
            elif i[1] in ['I-PER', 'I-ORG', 'I-LOC', 'B-LOC', 'B-ORG', 'B-PER', 'B-MISC', 'I-MISC']:
                word_temp += " " + i[0].upper()
                current_tag = i[1]
                count += 1
            old_tag = current_tag

In [ ]:
print(final_dict)
print_dict['Location']=list(set(final_dict['I-LOC']+final_dict['B-LOC']))
print_dict['Organisation']=list(set(final_dict['I-ORG']+final_dict['B-ORG']))
print_dict['Person Name']=list(set(final_dict['I-PER']+final_dict['B-PER']))
print_dict['Miscellaneous']=list(set(final_dict['I-MISC']+final_dict['B-MISC']))
print('')
print(print_dict)

In [ ]:
#sequence,tokens

In [51]:
locations=final_dict['I-LOC']+final_dict['B-LOC']
organisations=final_dict['I-ORG']+final_dict['B-ORG']
names=final_dict['I-PER']+final_dict['B-PER']
miscs=final_dict['I-MISC']+final_dict['B-MISC']

In [52]:
from collections import Counter
loc_count=Counter(locations)
org_count=Counter(organisations)
names_count=Counter(names)
miscs_count=Counter(miscs)

In [53]:
loc_key=loc_count.keys()
loc_value=loc_count.values()
org_key=org_count.keys()
org_value=org_count.values()
names_key=names_count.keys()
names_value=names_count.values()
miscs_key=miscs_count.keys()
miscs_value=miscs_count.values()

In [ ]:
height=10
width=15
plt.figure(figsize=(width,height))
plt.bar(loc_key,loc_value)
plt.title('Location with respect to Time', fontsize=35)
plt.xlabel('Location', fontsize=20)
plt.ylabel('Occurances', fontsize=20)
plt.xticks(rotation=90, fontsize=15)
plt.yticks(fontsize=15)
plt.show()

In [ ]:
height=20
width=50
plt.figure(figsize=(width,height))
plt.bar(names_key,names_value)
plt.title('Person names with respect to Time', fontsize=60)
plt.xlabel('Person Names', fontsize=50)
plt.ylabel('Occurances', fontsize=50)
plt.xticks(rotation=90, fontsize=30)
plt.yticks(fontsize=30)
plt.show()

In [ ]:
height=10
width=30
plt.figure(figsize=(width,height))
plt.bar(miscs_key,miscs_value)
plt.title('Miscellaneous with respect to Time', fontsize=50)
plt.xlabel('Miscellaneous', fontsize=40)
plt.ylabel('Occurances', fontsize=40)
plt.xticks(rotation=90, fontsize=20)
plt.yticks(fontsize=20)
plt.show()

In [ ]:
height=20
width=50
plt.figure(figsize=(width,height))
x=np.array(org_key)
y=np.array(org_value)
plt.barh(x, y)
plt.xticks(rotation=90,fontsize=25)
plt.yticks(fontsize=25)
plt.title('Organisation with respect to Time', fontsize=60)
plt.xlabel('Occurances',fontsize=40)
plt.ylabel('Organisation',fontsize=40)
plt.show()

In [ ]:
list_of_loc = final_dict['I-LOC']
#list_of_org = final_dict['I-ORG']
#list_of_person = final_dict['I-PER']
#list_of_misc = final_dict['I-MISC']
frequency = {}
for loc in list_of_loc:
  frequency[loc] = frequency.get(loc, 0) +1
  print(frequency)

In [ ]:
#outputs[0][6]

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([-0.7641342 , -2.4077516 , -2.0843105 , -3.2465644 ,  0.10769157,
       -2.4875414 ,  1.225174  , -1.9893391 ,  7.9986925 ], dtype=float32)>

In [ ]:
#label_list[8]

'I-LOC'

In [ ]:
#sequence,tokens

('I am Ajay from Chennai',
 ['[CLS]', 'I', 'am', 'A', '##jay', 'from', 'Chennai', '[SEP]'])

In [ ]:
#predictions.numpy()[0]

array([0, 0, 0, 4, 4, 0, 8, 0])